In [1]:
!pip install -q git+https://github.com/huggingface/transformers.git


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 596 kB 14.8 MB/s 
     |████████████████████████████████| 6.6 MB 65.5 MB/s 
     |████████████████████████████████| 101 kB 12.3 MB/s 


In [2]:
!pip install -q datasets==2.0.0

     |████████████████████████████████| 325 kB 14.7 MB/s 
     |████████████████████████████████| 212 kB 82.3 MB/s 
     |████████████████████████████████| 140 kB 78.1 MB/s 
     |████████████████████████████████| 1.1 MB 83.7 MB/s 
     |████████████████████████████████| 127 kB 90.3 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
     |████████████████████████████████| 144 kB 64.8 MB/s 
     |████████████████████████████████| 271 kB 68.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
!pip install -q seqeval


     |████████████████████████████████| 43 kB 1.6 MB/s 


In [4]:
!pip install -q pyyaml==5.1
# workaround: install old version of pytorch since detectron2 hasn't released packages for pytorch 1.9 (issue: https://github.com/facebookresearch/detectron2/issues/3158)
!pip install -q torch==1.11.0+cu113 torchvision==0.9.0+cu113 -f https://download.pytorch.org/whl/torch_stable.html

# install detectron2 that matches pytorch 1.8
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
#!pip install -q detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

#!git clone https://github.com/facebookresearch/detectron2.git
#!python -m pip install -e detectron2

     |████████████████████████████████| 274 kB 13.6 MB/s 
ERROR: Could not find a version that satisfies the requirement torchvision==0.9.0+cu113 (from versions: 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.2.2, 0.2.2.post2, 0.2.2.post3, 0.3.0, 0.4.0, 0.4.0+cpu, 0.4.0+cu92, 0.4.1, 0.4.1+cpu, 0.4.1+cu100, 0.4.1+cu92, 0.4.2, 0.4.2+cpu, 0.4.2+cu100, 0.4.2+cu92, 0.5.0, 0.5.0+cpu, 0.5.0+cu100, 0.5.0+cu92, 0.6.0, 0.6.0+cpu, 0.6.0+cu101, 0.6.0+cu92, 0.6.1, 0.6.1+cpu, 0.6.1+cu101, 0.6.1+cu92, 0.7.0, 0.7.0+cpu, 0.7.0+cu101, 0.7.0+cu92, 0.8.0, 0.8.1, 0.8.1+cpu, 0.8.1+cu101, 0.8.1+cu110, 0.8.1+cu92, 0.8.2, 0.8.2+cpu, 0.8.2+cu101, 0.8.2+cu110, 0.8.2+cu92, 0.9.0, 0.9.0+cpu, 0.9.0+cu101, 0.9.0+cu111, 0.9.1, 0.9.1+cpu, 0.9.1+cu101, 0.9.1+cu102, 0.9.1+cu111, 0.10.0, 0.10.0+cpu, 0.10.0+cu102, 0.10.0+cu111, 0.10.0+rocm4.1, 0.10.0+rocm4.2, 0.10.1, 0.10.1+cpu, 0.10.1+cu102, 0.10.1+cu111, 0.10.1+rocm4.1, 0.10.1+rocm4.2, 0.11.0, 0.11.0+cpu, 0.11.0+cu102, 0.11.0+cu111, 0.11.0+cu113, 0.11.0+rocm4.1, 0.11.0+roc

In [5]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 14.8 MB/s 


In [6]:
##imports
import os
import glob
import json 
import random




os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8" 
import torch 
import numpy as np
import random
import torch.nn as nn

from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
from dataclasses import dataclass
from dataclasses import fields
import cv2
import pandas as pd
from PIL import Image
from tqdm import tqdm
from IPython.display import display
import matplotlib
from PIL import Image
from matplotlib import pyplot, patches
from datasets import Features, Sequence, ClassLabel, Value, Array2D, Array3D
from datasets import load_dataset
from transformers import AutoProcessor,AutoModel
from torch.utils.data import DataLoader
from collections import OrderedDict, UserDict
from typing import Any, BinaryIO, ContextManager, Dict, List, Optional, Tuple, Union
from datasets import load_metric
from transformers import get_scheduler

from transformers import TrainingArguments, Trainer,LayoutLMv3Model, LayoutLMv3PreTrainedModel
from transformers.optimization import Adafactor, AdafactorSchedule, AdamW
from seqeval.metrics import (
    classification_report,
    f1_score,
    precision_score,
    recall_score,
)
seed = 7
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.use_deterministic_algorithms(True)



In [7]:

import spacy
nlp = spacy.load("en_core_web_sm")


In [8]:
# data loading


datasets = load_dataset("darentang/sroie",cache_dir=None)
#datasets = load_dataset("nielsr/funsd",cache_dir=None)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset sroie downloaded and prepared to /root/.cache/huggingface/datasets/darentang___sroie/sroie/1.0.0/26ed9374c9a15a1d2f44fd8886f679076e1a1fd7da2d53726d6e58a99436c506. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
#labels
labels = datasets['train'].features['ner_tags'].feature.names
print(labels)

['O', 'B-COMPANY', 'I-COMPANY', 'B-DATE', 'I-DATE', 'B-ADDRESS', 'I-ADDRESS', 'B-TOTAL', 'I-TOTAL']


In [10]:
#labels
id2label = {v: k for v, k in enumerate(labels)}
label2id = {k: v for v, k in enumerate(labels)}
label2id


{'B-ADDRESS': 5,
 'B-COMPANY': 1,
 'B-DATE': 3,
 'B-TOTAL': 7,
 'I-ADDRESS': 6,
 'I-COMPANY': 2,
 'I-DATE': 4,
 'I-TOTAL': 8,
 'O': 0}

In [11]:
pos2id= {'ADJ': 3,
 'ADP': 13,
 'ADV': 8,
 'AUX': 12,
 'CCONJ': 5,
 'DET': 7,
 'INTJ': 6,
 'NOUN': 15,
 'NUM': 0,
 'PART': 2,
 'PRON': 9,
 'PROPN': 1,
 'PUNCT': 14,
 'SYM': 10,
 'VERB': 4,
 'X': 11,
 'pad': 19,
 'CONJ':16,
 'SCONJ':17,
 'SPACE':18

 }

In [12]:

## preprocess the dataset


processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)
# we need to define custom features
features = Features({
    'pixel_values': Array3D(dtype="float64", shape=( 3, 224, 224)),
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'attention_mask': Sequence(Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'labels': Sequence(ClassLabel(names=labels)),
    "pos_tags":  Sequence(feature=Value(dtype='int64')),
    
})
# "pos_tags":  Sequence(feature=Value(dtype='int64')),

def get_tags(input_ids_batch):
  ## loop for each dcoment
  tags_examples = []
  for input_ids in input_ids_batch:

    y= processor.tokenizer.convert_ids_to_tokens(input_ids)
    words=[]
    for x in y:
      if x.startswith("##"):
        words[-1] = words[-1]+x.replace("##", "").strip()
      else:
        words.append(x)

    text = " ".join(words)
    text = text.replace("[CLS]", "").replace("[SEP]", "").replace("[PAD]", "").strip()
    doc = nlp(text)
    pos_tags = []
    for xx in doc:
      pos_tags.append(xx.pos_)
    count = 0
    total_pos = []
    for each in words:
      if each not in ["[CLS]", "[SEP]", "[PAD]"]:
        total_pos.append(pos_tags[count])
        count += 1
      else:
          total_pos.append("pad")
    assert len(words) == len(total_pos)

    def convert_to_original_length(y, tags):
   
        count = 0
        r_tags = []
        for index, token in enumerate(y):
            
            if token.startswith("##"):
                r_tags.append(r_tags[-1])
            else:
                r_tags.append(pos2id[tags[count]])
                count += 1
        return r_tags

    tags_examples.append(convert_to_original_length(y, total_pos))
  return tags_examples


    

def preprocess_data(examples):
  images = [Image.open(path).convert("RGB") for path in examples['image_path']]
  words = examples['words']
  boxes = examples['bboxes']
  word_labels = examples['ner_tags']
  
  encoded_inputs = processor(images, words, boxes=boxes, word_labels=word_labels,
                             padding="max_length", truncation=True)
  encoded_inputs["pos_tags"] = get_tags(encoded_inputs["input_ids"])
  # encoded_inputs = processor.tokenizer(images, words, boxes=boxes, word_labels=word_labels,
  #                            padding="max_length", truncation=True)
  return encoded_inputs
  
  
  # return encoded_inputs

# train_dataset = new_dataset.map(preprocess_data, batched=False, remove_columns=new_dataset.column_names,
#                                       features=features)
# test_dataset = new_dataset_test.map(preprocess_data, batched=True, remove_columns=new_dataset_test.column_names,
#                                       features=features)

# x = preprocess_data(datasets["train"][0:1])
train_dataset = datasets["train"].map(preprocess_data, batched=True, remove_columns=datasets["train"].column_names,
                                      features=features)
test_dataset = datasets["test"].map(preprocess_data, batched=True, remove_columns=datasets["test"].column_names,
                                       features=features)

Downloading:   0%|          | 0.00/275 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/856 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Parameter 'function'=<function preprocess_data at 0x7ff3405eee60> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [13]:
train_dataset.set_format(type="torch")
test_dataset.set_format(type="torch")

In [14]:

##creating a dataloader

from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=8)
test_dataloader = DataLoader(test_dataset, batch_size=8)

In [15]:
from collections import OrderedDict, UserDict
from typing import Any, BinaryIO, ContextManager, Dict, List, Optional, Tuple, Union

class ModelOutput(OrderedDict):
    """
    Base class for all model outputs as dataclass. Has a `__getitem__` that allows indexing by integer or slice (like a
    tuple) or strings (like a dictionary) that will ignore the `None` attributes. Otherwise behaves like a regular
    python dictionary.
    <Tip warning={true}>
    You can't unpack a `ModelOutput` directly. Use the [`~file_utils.ModelOutput.to_tuple`] method to convert it to a
    tuple before.
    </Tip>
    """

    def __post_init__(self):
        class_fields = fields(self)

        # Safety and consistency checks
        if not len(class_fields):
            raise ValueError(f"{self.__class__.__name__} has no fields.")
        if not all(field.default is None for field in class_fields[1:]):
            raise ValueError(f"{self.__class__.__name__} should not have more than one required field.")

        first_field = getattr(self, class_fields[0].name)
        other_fields_are_none = all(getattr(self, field.name) is None for field in class_fields[1:])

        if other_fields_are_none and not is_tensor(first_field):
            if isinstance(first_field, dict):
                iterator = first_field.items()
                first_field_iterator = True
            else:
                try:
                    iterator = iter(first_field)
                    first_field_iterator = True
                except TypeError:
                    first_field_iterator = False

            # if we provided an iterator as first field and the iterator is a (key, value) iterator
            # set the associated fields
            if first_field_iterator:
                for element in iterator:
                    if (
                        not isinstance(element, (list, tuple))
                        or not len(element) == 2
                        or not isinstance(element[0], str)
                    ):
                        break
                    setattr(self, element[0], element[1])
                    if element[1] is not None:
                        self[element[0]] = element[1]
            elif first_field is not None:
                self[class_fields[0].name] = first_field
        else:
            for field in class_fields:
                v = getattr(self, field.name)
                if v is not None:
                    self[field.name] = v

    def __delitem__(self, *args, **kwargs):
        raise Exception(f"You cannot use ``__delitem__`` on a {self.__class__.__name__} instance.")

    def setdefault(self, *args, **kwargs):
        raise Exception(f"You cannot use ``setdefault`` on a {self.__class__.__name__} instance.")

    def pop(self, *args, **kwargs):
        raise Exception(f"You cannot use ``pop`` on a {self.__class__.__name__} instance.")

    def update(self, *args, **kwargs):
        raise Exception(f"You cannot use ``update`` on a {self.__class__.__name__} instance.")

    def __getitem__(self, k):
        if isinstance(k, str):
            inner_dict = {k: v for (k, v) in self.items()}
            return inner_dict[k]
        else:
            return self.to_tuple()[k]

    def __setattr__(self, name, value):
        if name in self.keys() and value is not None:
            # Don't call self.__setitem__ to avoid recursion errors
            super().__setitem__(name, value)
        super().__setattr__(name, value)

    def __setitem__(self, key, value):
        # Will raise a KeyException if needed
        super().__setitem__(key, value)
        # Don't call self.__setattr__ to avoid recursion errors
        super().__setattr__(key, value)

    def to_tuple(self) -> Tuple[Any]:
        """
        Convert self to a tuple containing all the attributes/keys that are not `None`.
        """
        return tuple(self[k] for k in self.keys())



In [16]:
len(pos2id)

20

In [17]:

@dataclass
class TokenClassifierOutput(ModelOutput):
    """
    Base class for outputs of token classification models.
    Args:
        loss (`torch.FloatTensor` of shape `(1,)`, *optional*, returned when `labels` is provided) :
            Classification loss.
        logits (`torch.FloatTensor` of shape `(batch_size, sequence_length, config.num_labels)`):
            Classification scores (before SoftMax).
        hidden_states (`tuple(torch.FloatTensor)`, *optional*, returned when `output_hidden_states=True` is passed or when `config.output_hidden_states=True`):
            Tuple of `torch.FloatTensor` (one for the output of the embeddings + one for the output of each layer) of
            shape `(batch_size, sequence_length, hidden_size)`.
            Hidden-states of the model at the output of each layer plus the initial embedding outputs.
        attentions (`tuple(torch.FloatTensor)`, *optional*, returned when `output_attentions=True` is passed or when `config.output_attentions=True`):
            Tuple of `torch.FloatTensor` (one for each layer) of shape `(batch_size, num_heads, sequence_length,
            sequence_length)`.
            Attentions weights after the attention softmax, used to compute the weighted average in the self-attention
            heads.
    """

    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None





class LayoutLMv2ForTokenClassification2(LayoutLMv3PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.layoutlmv3 = AutoModel.from_pretrained("microsoft/layoutlmv3-base", output_hidden_states=True)  
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.embedding= nn.Embedding(21, 50, max_norm=True)
        ## for without lstm
        #self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        #self.lstm1 =  nn.LSTM(input_size=768, hidden_size=256, batch_first=True, bidirectional=True)
        #self.lstm2 =  nn.LSTM(input_size=256*2, hidden_size=256, batch_first=True, bidirectional=True)
        self.lstm3 =  nn.LSTM(input_size=768+50, hidden_size=128, batch_first=True, bidirectional=True)
        self.lstm4 =  nn.LSTM(input_size=128*2, hidden_size=128, batch_first=True, bidirectional=True)
        self.classifier = nn.Linear(128*2, config.num_labels)

        # Initialize weights and apply final processing
        #self.post_init()

    def get_input_embeddings(self):
        return self.layoutlmv2.embeddings.word_embeddings

   
    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        pos_tags: Optional[torch.LongTensor] = None,
        bbox: Optional[torch.LongTensor] = None,
        pixel_values: Optional[torch.FloatTensor] = None,
        attention_mask: Optional[torch.FloatTensor] = None,
        token_type_ids: Optional[torch.LongTensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, TokenClassifierOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size, sequence_length)`, *optional*):
            Labels for computing the token classification loss. Indices should be in `[0, ..., config.num_labels - 1]`.
        Returns:
        Examples:
        ```python
        >>> from transformers import LayoutLMv2Processor, LayoutLMv2ForTokenClassification
        >>> from PIL import Image
        >>> processor = LayoutLMv2Processor.from_pretrained("microsoft/layoutlmv2-base-uncased", revision="no_ocr")
        >>> model = LayoutLMv2ForTokenClassification.from_pretrained("microsoft/layoutlmv2-base-uncased")
        >>> image = Image.open("name_of_your_document - can be a png file, pdf, etc.").convert("RGB")
        >>> words = ["hello", "world"]
        >>> boxes = [[1, 2, 3, 4], [5, 6, 7, 8]]  # make sure to normalize your bounding boxes
        >>> word_labels = [0, 1]
        >>> encoding = processor(image, words, boxes=boxes, word_labels=word_labels, return_tensors="pt")
        >>> outputs = model(**encoding)
        >>> loss = outputs.loss
        >>> logits = outputs.logits
        ```"""
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        pos_output = self.embedding(pos_tags)
        outputs = self.layoutlmv3(
            input_ids=input_ids,

            bbox=bbox,
            pixel_values=pixel_values,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        if input_ids is not None:
            input_shape = input_ids.size()
        else:
            input_shape = inputs_embeds.size()[:-1]

        seq_length = input_shape[1]
        # only take the text part of the output representations
        
        # sequence_output = outputs[0][:, :seq_length]
        # print(sequence_output.shape)
        # sequence_output = self.dropout(sequence_output)
        # print(sequence_output.shape)
        # logits = self.classifier(sequence_output)

        # uncomment for bilstm
        # sequence_output = outputs[0][:, :seq_length]
        #print(len(outputs[1]))
        sequence_output = torch.stack(list(outputs[1]), dim=0)
        sequence_output=torch.mean(sequence_output,0)[:, :seq_length]
        sequence_output = torch.cat([sequence_output,pos_output], -1)
        # print(sequence_output.shape)
        
        # sequence_output = self.dropout(sequence_output)
        sequence_output, _ = self.lstm3(sequence_output)

       
        # sequence_output, _ = self.lstm4(sequence_output)
 
        sequence_output, _ = self.lstm4(sequence_output)
        
        sequence_output = self.dropout(sequence_output)
        #print(sequence_output.shape)

        logits = self.classifier(sequence_output)

        loss = None
        if labels is not None:
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return TokenClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )


In [18]:
os.environ["CUDA_LAUNCH_BLOCKING"]="1"

In [19]:

from datasets import load_metric
import json
from transformers import get_scheduler

model = AutoModel.from_pretrained("microsoft/layoutlmv3-base", output_hidden_states=True)  
model.config.id2label = id2label
model.config.label2id = label2id
model.config.num_labels = len(id2label)
models = LayoutLMv2ForTokenClassification2(model.config)
for p in models.layoutlmv3.parameters():
  p.requires_grad = False

# Set id2label and label2id 


# Metrics
metric = load_metric("seqeval")
return_entity_level_metrics = True

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return json.dump({
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        })



class FunsdTrainer(Trainer):
    def get_train_dataloader(self):
      return train_dataloader

    def get_test_dataloader(self, test_dataset):
      return test_dataloader

from transformers.optimization import Adafactor, AdafactorSchedule, AdamW
#optimizer = Adafactor(model.parameters(), scale_parameter=True, relative_step=True, warmup_init=True, lr=None)

optimizer = AdamW(params=models.parameters(), lr=0.006)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=2000,
)
args = TrainingArguments(
    
    output_dir="layoutlmv2-finetuned-funsd-v2", # name of directory to store the checkpoints
    max_steps=2000, # we train for a maximum of 1,000 batches
    warmup_ratio=0.4, # we warmup a bit
    fp16=True
)

# Initialize our Trainer
trainer = FunsdTrainer(
    model=models,
    args=args,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, lr_scheduler)
)


Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
max_steps is given, it will override any value given in num_train_epochs
Using cuda_amp half precision backend


In [20]:
trainer.train()

# predictions, labels, metrics = trainer.predict(test_dataset)


***** Running training *****
  Num examples = 626
  Num Epochs = 26
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2000
/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:703: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  "The `device` argument is deprecated and will be removed in v5 of Transformers.", FutureWarning


Step,Training Loss
500,0.086500
1000,0.022100
1500,0.007800
2000,0.002100


Saving model checkpoint to layoutlmv2-finetuned-funsd-v2/checkpoint-500
Configuration saved in layoutlmv2-finetuned-funsd-v2/checkpoint-500/config.json
Model weights saved in layoutlmv2-finetuned-funsd-v2/checkpoint-500/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:703: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  "The `device` argument is deprecated and will be removed in v5 of Transformers.", FutureWarning
Saving model checkpoint to layoutlmv2-finetuned-funsd-v2/checkpoint-1000
Configuration saved in layoutlmv2-finetuned-funsd-v2/checkpoint-1000/config.json
Model weights saved in layoutlmv2-finetuned-funsd-v2/checkpoint-1000/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:703: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  "The `device` argument is deprecated and will be removed in v5 of Transformers.", F

TrainOutput(global_step=2000, training_loss=0.029620989561080932, metrics={'train_runtime': 634.1219, 'train_samples_per_second': 25.232, 'train_steps_per_second': 3.154, 'total_flos': 4244508166502400.0, 'train_loss': 0.029620989561080932, 'epoch': 25.32})

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from seqeval.metrics import (
    classification_report,
    f1_score,
    precision_score,
    recall_score,
)

In [23]:



eval_loss = 0.0
nb_eval_steps = 0
preds = None
out_label_ids = None

# put model in evaluation mode
models.eval()
for batch in tqdm(test_dataloader, desc="Evaluating"):
    with torch.no_grad():
        print(batch.keys())
        input_ids = batch["input_ids"].to(device)
        bbox = batch["bbox"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        image = batch['pixel_values'].to(device)
        pos_tags=batch["pos_tags"].to(device)

        # forward pass
        outputs = models(pos_tags=pos_tags,pixel_values = image , input_ids=input_ids, bbox=bbox, attention_mask=attention_mask,labels=labels)
        # get the loss and logits
        tmp_eval_loss = outputs.loss
        logits = outputs.logits

        eval_loss += tmp_eval_loss.item()
        nb_eval_steps += 1

        # compute the predictions
        if preds is None:
            preds = logits.detach().cpu().numpy()
            out_label_ids = labels.detach().cpu().numpy()
        else:
            preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(
                out_label_ids, labels.detach().cpu().numpy(), axis=0
            )

# compute average evaluation loss
eval_loss = eval_loss / nb_eval_steps
preds = np.argmax(preds, axis=2)

out_label_list = [[] for _ in range(out_label_ids.shape[0])]
preds_list = [[] for _ in range(out_label_ids.shape[0])]

for i in range(out_label_ids.shape[0]):
    for j in range(out_label_ids.shape[1]):
        if out_label_ids[i, j] != -100:
            out_label_list[i].append(id2label[out_label_ids[i][j]])
            preds_list[i].append(id2label[preds[i][j]])

results = {
    "loss": eval_loss,
    "precision": precision_score(out_label_list, preds_list),
    "recall": recall_score(out_label_list, preds_list),
    "f1": f1_score(out_label_list, preds_list),
}
print(results)

Evaluating:   0%|          | 0/44 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:703: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  "The `device` argument is deprecated and will be removed in v5 of Transformers.", FutureWarning


dict_keys(['pixel_values', 'input_ids', 'attention_mask', 'bbox', 'labels', 'pos_tags'])


Evaluating:   2%|▏         | 1/44 [00:00<00:15,  2.85it/s]

dict_keys(['pixel_values', 'input_ids', 'attention_mask', 'bbox', 'labels', 'pos_tags'])


Evaluating:   5%|▍         | 2/44 [00:00<00:13,  3.12it/s]

...

dict_keys(['pixel_values', 'input_ids', 'attention_mask', 'bbox', 'labels', 'pos_tags'])


Evaluating:  95%|█████████▌| 42/44 [00:12<00:00,  3.31it/s]

dict_keys(['pixel_values', 'input_ids', 'attention_mask', 'bbox', 'labels', 'pos_tags'])


Evaluating: 100%|██████████| 44/44 [00:13<00:00,  3.35it/s]


dict_keys(['pixel_values', 'input_ids', 'attention_mask', 'bbox', 'labels', 'pos_tags'])
{'loss': 0.03791845258853176, 'precision': 0.8942101501072195, 'recall': 0.9012968299711815, 'f1': 0.8977395048439182}
